In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import math
#readingsourcefile
filename = "attack"
df = pd.read_csv(filename, sep =',' , header = None, error_bad_lines = False)
# removing first row of data then sorting according to time field
array = df.to_numpy()
#removing protocol column
arr = np.delete(array, 5, axis=0)
arr = np.delete(array, 0, axis=0)
array = sorted(arr, key=lambda x: float(x[1]))
array

In [ ]:
# storing each packet's transmission duration
packetDurations = []
# naming values of info field
infoList = ['RPL Control (Destination Advertisement Object)',
'RPL Control (DODAG Information Solicitation)',
'RPL Control (DODAG Information Object)', 'Ack',
'PDUType: 108 t Unknown',
'PDUType: 112 t Unknown',
'3000 > 3001 Len=11']
infoDict = {}
index = 1
for info in infoList:
  infoDict[info] = index
  index += 1
#print(infoDict)
# storing malicious nodes
labelNodes = []
# rewriting source, destination and info columns #
counter = 0

In [ ]:
counter = 0
while(counter < len(array)):
  src = array[counter][2]
  dst = array[counter][3]
  array[counter][2] = int(src.split(":").pop(), 16) if not pd.isnull(src) else src
  if(dst == 'ff02::1a'):
    array[counter][3] = int(9999) if not pd.isnull(dst) else dst
  else:
    array[counter][3] = int(dst.split(":").pop(), 16) if not pd.isnull(dst) else dst
  if array[counter][6] in infoDict:
    array[counter][6] = infoDict[array[counter][6]]
  else:
    array[counter][6]=None
  if counter != 0 and counter + 1 < len(array):
    duration = float(array[counter][1]) - float(array[counter - 1][1])
    packetDurations.append([math.floor(float(array[counter][1])),array[counter][2],array[counter][3], duration])

  counter += 1

In [ ]:
# Calculating node counts in per seconds #
currentSecond = 0.0
lastSecond = math.floor(float(array[-1][1]))
packetsInEachSecond = dict()
while currentSecond <= lastSecond:
  currentSecondArray = []
  currentRow = 1
  while currentRow < len(array):
    currentPacketSecond = floor(float(array[currentRow][1]))
    if currentPacketSecond == currentSecond:
      currentSecondArray.append(array[currentRow])
    currentRow += 1
  packetsInEachSecond[currentSecond] = currentSecondArray
  currentSecond += 1.0

NameError: ignored

In [ ]:
# Calculating control packet counts in per seconds #
sourceCountsBySeconds = dict()
destinationCountsBySeconds =dict()
daoCountsBySeconds =dict()
disCountsBySeconds =dict()
dioCountsBySeconds =dict()
totalDaoDisDioBySecond =dict()
for sec in packetsInEachSecond:
  sourceNodeCounts = dict()
  destinationNodeCounts =dict()
  daoCountsByNode =dict()
  disCountsByNode =dict()
  dioCountsByNode =dict()
  for packet in packetsInEachSecond[sec]:
    src = packet[2]
    dst = packet[3]
    info = packet[5]
    sourceNodeCounts[src] = 1 if src not in sourceNodeCounts else sourceNodeCounts[src] + 1
    destinationNodeCounts[dst] = 1 if dst not in destinationNodeCounts else destinationNodeCounts[dst] + 1
    if info == 1:
      daoCountsByNode[src] = 1 if src not in daoCountsByNode else daoCountsByNode[src] + 1
    elif info == 2:
      disCountsByNode[src] = 1 if src not in disCountsByNode else disCountsByNode[src] + 1
    elif info == 3:
      dioCountsByNode[src] = 1 if src not in dioCountsByNode else dioCountsByNode[src] + 1
    if info == 1 or 2 or 3
      totalDaoDisDioBySecond[sec] = 1 if sec not in totalDaoDisDioBySecond else totalDaoDisDioBySecond[sec] + 1
  sourceCountsBySeconds[sec] = sourceNodeCounts
  destinationCountsBySeconds[sec] = destinationNodeCounts
  daoCountsBySeconds[sec] = daoCountsByNode
  disCountsBySeconds[sec] = disCountsByNode
  dioCountsBySeconds[sec] = dioCountsByNode

In [ ]:
# Calculating total duration times by seconds #
transTotalDurBySec =dict()
for sec in sourceCountsBySeconds:
  transTotalDurBySec[sec] =dict()
  for node in sourceCountsBySeconds[sec]:
    transTotalDurBySec[sec][node] = 0
rcvTotalDurBySec =dict()
for sec in destinationCountsBySeconds:
  rcvTotalDurBySec[sec] =dict()
  for node in destinationCountsBySeconds[sec]:
    rcvTotalDurBySec[sec][node] = 0
counter = 1
while counter < len(packetDurations):
  second = packetDurations[counter][0]
  try:
    nodeTr = packetDurations[counter][1]
    nodeRcv = packetDurations[counter][2]
    transTotalDurBySec[second][nodeTr] = transTotalDurBySec[second][nodeTr] + packetDurations[counter][3]
    rcvTotalDurBySec[second][nodeRcv] = rcvTotalDurBySec[second][nodeRcv] + packetDurations[counter][3]
  except:
    print("This is an error message!")
  counter += 1

In [ ]:
# Creating enriched IoT dataset file #
monitorArray = []
counter = 0
labelCounter = 0
while counter < len(array):
  if not pd.isnull(array[counter][2]):
    src = array[counter][2]
    dst = array[counter][3]
    row = array[counter]
    second = floor(float(array[counter][1]))
    srcCount = 0.0
    dstCount = 0.0
    if second in sourceCountsBySeconds:
      srcCounts = sourceCountsBySeconds[second]
      if src in srcCounts:
        srcCount = srcCounts[src]
    if second in destinationCountsBySeconds:
      dstCounts = destinationCountsBySeconds[second]
      if dst in dstCounts:
        dstCount = dstCounts[dst]
    transmissionRate = srcCount / 1000.0
    receptionRate = dstCount / 1000.0
    TrRr = transmissionRate / receptionRate if receptionRate != 0 else 0
    trnTtlDur = transTotalDurBySec[second][src]
    rcvTtlDur = rcvTotalDurBySec[second][dst]
    trnAverageTime = trnTtlDur / srcCount
    rcvAverageTime = rcvTtlDur / dstCount
    daoSrcCount = 0
    disSrcCount = 0
    dioSrcCount = 0
    if len(daoCountsBySeconds[second]) > 0 and src in daoCountsBySeconds[second]:
      daoSrcCount = daoCountsBySeconds[second][src]
    else:
      daoSrcCount = 0
    if len(disCountsBySeconds[second]) > 0 and src in disCountsBySeconds[second]:
      disSrcCount = disCountsBySeconds[second][src]
    else:
      disSrcCount = 0
    if len(dioCountsBySeconds[second]) > 0 and src in dioCountsBySeconds[second]:
      dioSrcCount = dioCountsBySeconds[second][src]
    else:
      dioSrcCount = 0
    dao = daoSrcCount
    dis = disSrcCount
    dio = dioSrcCount
    label = 1
    row = np.append(row, str(transmissionRate))
    row = np.append(row, str(receptionRate))
    row = np.append(row, str(TrRr))
    row = np.append(row, str(srcCount))
    row = np.append(row, str(dstCount))
    row = np.append(row, str(trnTtlDur))
    row = np.append(row, str(rcvTtlDur))
    row = np.append(row, str(trnAverageTime))
    row = np.append(row, str(rcvAverageTime))
    row = np.append(row, str(dao))
    row = np.append(row, str(dis))
    row = np.append(row, str(dio))
    row = np.append(row, str(label))
    monitorArray.append(row)
  counter += 1

In [ ]:
headers = ['No.', 'Time', 'Source', 'Destination', 'Length', 'Info',
'Transmission Rate (per 1000 ms)',
'Reception Rate (per 1000 ms)',
'TR / RR',
'Sources Count Per Sec',
'Destinations Count Per Sec',
'Trans Total Duration Per Sec',
'Rcv Total Duration Per Sec',
'Trans Average Per Sec',
'Rcv Average Per Sec',
'DAO',
'DIS',
'DIO',
'Label'
]
monitorArray.insert(0, headers)
with open('result.csv', 'w') as monitoring:
  wr = csv.writer(monitoring, dialect='excel', delimiter=',')
  wr.writerows(monitorArray)

In [ ]:
##########################################
##########################################
# Script 2 - Data Normalization Algorithm #
##########################################
# import the required packages
#import pandas as pd
#...
# dataset headers
#path1='./csvs/'
headers_val = [
'Length', 'Info',
'Transmission Rate (per 1000 ms)',
'Reception Rate (per 1000 ms)',
'TR / RR',
'Sources Count Per Sec',
'Destinations Count Per Sec',
'Trans Total Duration Per Sec',
'Rcv Total Duration Per Sec',
'Trans Average Per Sec',
'Rcv Average Per Sec',
'DAO',
'DIS',
'DIO'
]
amgPd = pd.DataFrame()
# import dataset
for chunk in pd.read_csv('result.csv', chunksize = 250000, low_memory=False):
  amgPd = pd.concat([amgPd,chunk])
print ('result')
print (amgPd.describe())

In [ ]:
# transform matrix format
A = amgPd.as_matrix()
row_num = A.shape[1]
X = A[:, 0:row_num - 1]
Y = A[:, row_num - 1].astype(int)
# feature normalization #
X = preprocessing.quantile_transform (X, output_distribution='normal')
X = preprocessing.minmax_scale (X)
data = pd.DataFrame(X, columns=headers_val)
labels = pd.DataFrame(Y)

In [ ]:
# creating main dataset #
amgPd = data.join(labels.rename(columns=0:'Label'))
print ('BH1000%5mix after scaling')
print (amgPd.describe())
# transform to csv of normalized dataset
amgPd.to_csv('BH1000%5mix_norm.csv', index=False)
# import another dataset
amgPd1 = pd.DataFrame()
for chunk in pd.read_csv(path1+'BH100%10mix.csv', chunksize = 250000, low_memory=False):
  amgPd1 = pd.concat([amgPd1,chunk])
amgPd1.drop([u'No.','Time', u'Source', u'Destination'], axis=1, inplace=True)
print ('BH100%10')
print (amgPd1.describe())
# transform matrix format
A = amgPd1.as_matrix()
row_num = A.shape[1]
X = A[:, 0:row_num - 1]
Y = A[:, row_num - 1].astype(int)
# feature normalization #
X = preprocessing.quantile_transform (X, output_distribution='normal')
X = preprocessing.minmax_scale (X)
data = pd.DataFrame(X, columns=headers_val)
labels = pd.DataFrame(Y)
# concatenating the datasets #
amgPd1 = data.join(labels.rename(columns=0:'Label'))
print ('BH100%10 after scaling')
print (amgPd1.describe())
amgPd1.to_csv('BH100%10mix_norm.csv', index=False)
# appending to main dataset
amgPd = pd.concat ([amgPd, amgPd1])